# Finetuning a Convolutional Neural Network

In this exercise, you will have to finetune a pretrained CNN model on the CIFAR10 dataset. The data loading and model testing logic are already included in your code. You will have to create the model and the training loop.

**In this workspace you have GPU to help train the model but it is best practice to DISABLE it while writing code and only ENABLE it when you are training.** 

Here are the steps you need to do to complete this exercise:

1. Finish the `create_model()` function. You should use a pretrained model. You are free to choose any pre-trained model that you want to use. 
2. Finish the `train()` function. This function should validate the accuracy of the model during the training stage. You should stop the training when this validation accuracy stops increasing.
3. Save all your work and then **ENABLE** the GPU
4. Run the file to make sure that the model is training properly.
5. If it works, remember to **DISABLE** the GPU before moving to the next page. 

In case you get stuck, you can look at the solution by clicking the jupyter symbol at the top left and navigating to `finetune_a_cnn_solution.py`.

## Try It Out!
- See how your accuracy changes when using other pre-trained models.
- Play around with the number of layers and neurons in your model. How does the accuracy change? How long does it take to train the model?
- Can you create the same network in TensorFlow as well?

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import time

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Running on Device {device}")

Running on Device cuda:0


In [14]:

def test(model, test_loader, criterion):
    model.eval()
    running_loss=0
    running_corrects=0
    
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs=model(inputs)
        loss=criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data).item()

    total_loss = running_loss / len(test_loader)
    total_acc = running_corrects / len(test_loader)

    print(f"Test Loss: {total_loss}, Test Accuracy: {total_acc}")
    return total_loss, total_acc
    
def train(model, train_loader, validation_loader, criterion, optimizer):
    epochs=10
    best_loss=1e6
    image_dataset={'train':train_loader, 'valid':validation_loader}
    loss_counter=0
    
    for epoch in range(epochs):
        for phase in ['train', 'valid']:
            print(f"Epoch {epoch}, Phase {phase}")
            if phase=='train':
                model.train()
            else:
                model.eval()
        #TODO: Finish the rest of the training code
        # The code should stop training when the validation accuracy
        # stops increasing
        # You should save the model with the best validation accuracy
        # and return the best model
        running_loss=0
        running_corrects=0
        for inputs, labels in image_dataset[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase=='train'):
                outputs=model(inputs)
                loss=criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)
                
                if phase=='train':
                    loss.backward()
                    optimizer.step()
                    
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data).item()
                
        epoch_loss = running_loss / len(image_dataset[phase].dataset)
        epoch_acc = running_corrects / len(image_dataset[phase].dataset)
        
        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
    return model

def create_model():
    # TODO: Write code to create the model
    model = models.resnet18(pretrained=True)
    
    for param in model.parameters():
        param.requires_grad = False
        
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 10)

    model = model.to(device)  # Add this line


    return model




In [10]:
batch_size=32


training_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

testing_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
        download=True, transform=training_transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
        shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
        download=True, transform=testing_transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
        shuffle=False)



Files already downloaded and verified
Files already downloaded and verified


In [11]:

start_time = time.time()

model=create_model()
model=model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

train(model, trainloader, testloader, criterion, optimizer)

end_time = time.time()
print(f"Total time taken: {end_time - start_time}")


Epoch 0, Phase train
Epoch 0, Phase valid
valid Loss: 2.6694 Acc: 0.0738
Epoch 1, Phase train
Epoch 1, Phase valid
valid Loss: 2.6694 Acc: 0.0738
Epoch 2, Phase train
Epoch 2, Phase valid
valid Loss: 2.6694 Acc: 0.0738
Epoch 3, Phase train
Epoch 3, Phase valid
valid Loss: 2.6694 Acc: 0.0738
Epoch 4, Phase train
Epoch 4, Phase valid
valid Loss: 2.6694 Acc: 0.0738
Epoch 5, Phase train
Epoch 5, Phase valid
valid Loss: 2.6694 Acc: 0.0738
Epoch 6, Phase train
Epoch 6, Phase valid
valid Loss: 2.6694 Acc: 0.0738
Epoch 7, Phase train
Epoch 7, Phase valid
valid Loss: 2.6694 Acc: 0.0738
Epoch 8, Phase train
Epoch 8, Phase valid
valid Loss: 2.6694 Acc: 0.0738
Epoch 9, Phase train
Epoch 9, Phase valid
valid Loss: 2.6694 Acc: 0.0738
Total time taken: 112.72793078422546


In [15]:
test(model, testloader, criterion)

Test Loss: 85.28436913048498, Test Accuracy: 2.357827476038339


(85.28436913048498, 2.357827476038339)

## Remember to Disable GPU when you are done training. 